# Imports

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
from scipy.stats import norm
import os.path as osp
import torch
import xgboost as xgb

# For using the mgbdt library, you have to include the library directory into your python path.
# If you are in this repository's root directory, you can do it by using the following lines
import sys,os
# sys.path.insert(0, os.path.join('mGBDT','lib'))
sys.path.insert(0, os.path.join('lib'))

from mgbdt import MGBDT
from mgbdt import MultiXGBModel, LinearModel
from mgbdt.utils.plot_utils import plot2d, plot3d
from mgbdt.utils.exp_utils import set_seed
from mgbdt.utils.log_utils import logger

# Data

In [2]:
# boston = datasets.load_boston()
# X, y = boston.data, boston.target

import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
y = raw_df.values[1::2, 2]

x_mean = X.mean(axis=0)
x_std = X.std(axis=0)
X = (X - x_mean) / x_std

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Train Models

In [3]:
# Hyper-Parameters
num_boost_round = 5
learning_rate = 0.03
max_depth = 5
loss = "L1Loss"

n_epochs = 200

## mGBDT Linear

In [4]:
np.random.seed(123)
torch.manual_seed(123)

net_linear = MGBDT(loss=None, target_lr=1, epsilon=0.1, verbose=False) 

# Add target-propogation layers: F, G represent the forward and inverse mapping layers, respectively
net_linear.add_layer("tp_layer",
                     F=MultiXGBModel(input_size=X_train.shape[1], output_size=5, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round),
                     G=None)
net_linear.add_layer("tp_layer",
                     F=MultiXGBModel(input_size=5, output_size=3, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round),
                     G=MultiXGBModel(input_size=3, output_size=5, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round))
net_linear.add_layer("bp_layer",
                     F=LinearModel(input_size=3, output_size=1, learning_rate=0.01, loss=loss))


# init the forward mapping
net_linear.init(X_train, n_rounds=10)

net_linear.fit(X_train, y_train.reshape(-1,1), n_epochs=n_epochs) 

d:\miniforge3\envs\mdgbt\lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
d:\miniforge3\envs\mdgbt\lib\site-packages\xgboost\core.py:613: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn("Use subset (sliced data) of np.ndarray is not recommended " +
[ 2025-02-07 18:23:22,856][mgbdt.log] [epoch=0/200][train] loss=602.797764
[ 2025-02-07 18:23:23,185][mgbdt.log] [epoch=1/200][train] loss=601.589081
[ 2025-02-07 18:23:23,491][mgbdt.log] [epoch=2/200][train] loss=600.068914
[ 2025-02-07 18:23:23,798][mgbdt.log] [epoch=3/200][train] loss=598.612313
[ 2025-02-07 18:23:24,115][mgbdt.log] [epoch=4/200][train] loss=597.232585
[ 2025-02-07 18:23:24,432][mgbdt.log] [epoch=5/200][train] loss=595.811719
[ 2025-02-07 18:23:24,751][mgbdt.log] [epoch=6/200][train] lo

## mGBDT XGBoost

In [5]:
np.random.seed(123)
torch.manual_seed(123)

net_xgboost = MGBDT(loss=loss, target_lr=1, epsilon=0.1, verbose=False) 

# Add target-propogation layers: F, G represent the forward and inverse mapping layers, respectively
net_xgboost.add_layer("tp_layer",
                      F=MultiXGBModel(input_size=X_train.shape[1], output_size=5, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round),
                      G=None)
net_xgboost.add_layer("tp_layer",
                      F=MultiXGBModel(input_size=5, output_size=3, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round),
                      G=MultiXGBModel(input_size=3, output_size=5, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round))
net_xgboost.add_layer("tp_layer",
                      F=MultiXGBModel(input_size=3, output_size=1, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round),
                      G=MultiXGBModel(input_size=1, output_size=3, learning_rate=learning_rate, max_depth=max_depth, num_boost_round=num_boost_round))


# init the forward mapping
net_xgboost.init(X_train, n_rounds=10)

net_xgboost.fit(X_train, y_train.reshape(-1,1), n_epochs=n_epochs) 

d:\miniforge3\envs\mdgbt\lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
[ 2025-02-07 18:25:46,587][mgbdt.log] [epoch=0/200][train] loss=22.386312
[ 2025-02-07 18:25:47,061][mgbdt.log] [epoch=1/200][train] loss=22.244833
[ 2025-02-07 18:25:47,512][mgbdt.log] [epoch=2/200][train] loss=22.115596
[ 2025-02-07 18:25:48,120][mgbdt.log] [epoch=3/200][train] loss=21.971816
[ 2025-02-07 18:25:48,590][mgbdt.log] [epoch=4/200][train] loss=21.832660
[ 2025-02-07 18:25:49,040][mgbdt.log] [epoch=5/200][train] loss=21.691416
[ 2025-02-07 18:25:49,492][mgbdt.log] [epoch=6/200][train] loss=21.550116
[ 2025-02-07 18:25:49,958][mgbdt.log] [epoch=7/200][train] loss=21.408705
[ 2025-02-07 18:25:50,451][mgbdt.log] [epoch=8/200][train] loss=21.266461
[ 2025-02-07 18:25:50,903][mgbdt.log] [epoch=9/200][train] loss=21.123349
[ 2025-02-07 18:25:51,421][mgbdt.log] [epoch=10/200][tr

# Linear Model

In [6]:
torch.manual_seed(123)

linear_model = LinearModel(input_size=13, output_size=1, loss=loss)
for _ in range(n_epochs):
    linear_model.fit(X_train, y_train)

d:\miniforge3\envs\mdgbt\lib\site-packages\torch\nn\modules\loss.py:128: UserWarning: Using a target size (torch.Size([354])) that is different to the input size (torch.Size([354, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


## XGBoost

In [7]:
np.random.seed(123)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

params = {"eta": learning_rate,                   
          "max_depth": max_depth
         }

xgb_model = xgb.train(params,
                      dtrain,
                      num_boost_round=n_epochs)

# Predictions

In [8]:
pred_mgbdt_linear = pd.DataFrame.from_dict({"model": "mGBDT_Linear",
                                            "pred": net_linear.forward(X_test).reshape(-1,),
                                            "y": y_test
                                           })

pred_mgbdt_xgboost = pd.DataFrame.from_dict({"model": "mGBDT_XGBoost", 
                                             "pred": net_xgboost.forward(X_test).reshape(-1,),
                                             "y": y_test
                                            })

pred_linear = pd.DataFrame.from_dict({"model": "Linear", 
                                      "pred": linear_model.predict(X_test).reshape(-1,),
                                      "y": y_test
                                     })


pred_xgboost = pd.DataFrame.from_dict({"model": "XGBoost",
                                       "pred": xgb_model.predict(dtest),
                                       "y": y_test
                                      })

pred_df = pd.concat([pred_mgbdt_linear, pred_mgbdt_xgboost, pred_linear, pred_xgboost])

# Evaluation

In [9]:
pred_df.groupby("model").apply(lambda x: mean_absolute_error(y_true=x.y, y_pred=x.pred)).reset_index().rename(columns={0: "MAE"}).sort_values(by="MAE").reset_index(drop=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_45052\1387413843.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pred_df.groupby("model").apply(lambda x: mean_absolute_error(y_true=x.y, y_pred=x.pred)).reset_index().rename(columns={0: "MAE"}).sort_values(by="MAE").reset_index(drop=True)


,model,MAE
0,XGBoost,2.377509
1,mGBDT_Linear,3.703030
2,mGBDT_XGBoost,6.272753
3,Linear,72.753305
